# Documentation for Python Script: Predicting Energy Demand

This Python script demonstrates how to collect historical energy demand data, store it in an Azure SQL Database, preprocess the data, and use a Random Forest Regressor model to predict energy demand for the next 24 hours. It also connects to an external website to scrape data, uses BeautifulSoup for web scraping, and leverages the pyodbc library for database interactions.


## Execute the script. It will perform the following steps:

1. Connect to the Azure SQL Database.
2. Create the required table.
3. Scrape historical energy demand data from a website.
4. Insert the scraped data into the database.
5. Fetch historical demand and weather data from the database.
6. Preprocess the data.
7. Train a Random Forest Regressor model and make predictions.
8. Calculate and display MAE and RMSE.
9. Disconnect from the database.

### Prerequisites

Before running this script, make sure you have the following prerequisites in place:

1. **Python**: You need Python installed on your system. You can download and install Python from the [official Python website](https://www.python.org/downloads/).

2. **Required Libraries**: Install the necessary Python libraries using `pip`:
```
pip install pyodbc requests beautifulsoup4 python-dateutil pandas scikit-learn
```
   
### Usage

**Constants and Parameters**: At the beginning of the script specify constants and parameters, including database connection details (server, database, username, password), and the ODBC driver used for connecting to the Azure SQL Database.

```python
server = "hdcom1ser.database.windows.net"
database = "hdcom1"
username = "serveradminhd"
password = "A@123456a"
driver = "{ODBC Driver 18 for SQL Server}"
```

**DatabaseConnection Class**: The DatabaseConnection class handles database connection, disconnection, and executing SQL queries.

        - connect(): Connects to the Azure SQL Database using the provided credentials.
        - disconnect(): Disconnects from the database.
        - execute(sql, params=None): Executes an SQL query, optionally with parameters.

**create_azure_sql_table Function**: This function creates a table in the Azure SQL Database if it doesn't exist. Modify the table schema as needed.

**scrape_data_to_list Function**: This function scrapes historical energy demand data from a website and returns it as a list.

**insert_data_to_azure_sql Function**: This function inserts scraped data into the Azure SQL Database table.

**fetch_data_from_azure_sql Function**: This function retrieves data from the Azure SQL Database based on a specified time range.

**preprocess_data Function**: This function preprocesses the fetched data, including converting timestamps, adding day of the week and hour columns, and merging with weather data.

**train_and_predict_model Function**: This function trains a Random Forest Regressor model and predicts energy demand for the next 24 hours. It also calculates mean absolute error (MAE) and root mean squared error (RMSE).

**Main Execution Flow**: The main execution flow of the script connects to the database, creates the required table, scrapes data, inserts it into the database, fetches data, preprocesses it, trains the model, makes predictions, and disconnects from the database.
Disconnect from the Database: Ensure you disconnect from the Azure SQL Database when you are done.



In [ ]:
import pyodbc
import requests
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, timedelta

# Constants and Parameters
server = "hdcom1ser.database.windows.net"
database = "hdcom1"
username = "serveradminhd"
password = "A@123456a"
driver = "{ODBC Driver 18 for SQL Server}"
base_url = "http://ets.aeso.ca/ets_web/ip/Market/Reports/ActualForecastWMRQHReportServlet"
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};ncrypt=yes;TrustServerCertificate=no'


class DatabaseConnection:
    def __init__(self):
        self.conn_str = conn_str
        self.conn = None

    def connect(self):
        try:
            self.conn = pyodbc.connect(self.conn_str)
        except Exception as e:
            print(f"Error connecting to the database: {e}")

    def disconnect(self):
        if self.conn:
            self.conn.close()
            self.conn = None

    def execute(self, sql, params=None):
        try:
            cursor = self.conn.cursor()
            if params:
                cursor.execute(sql, params)
            else:
                cursor.execute(sql)
            self.conn.commit()
        except Exception as e:
            print(f"Error executing SQL query: {e}")

def create_azure_sql_table(connection):
    # Drop the table if it exists
    drop_table_query = f"IF OBJECT_ID('historical_demand_data', 'U') IS NOT NULL BEGIN DROP TABLE historical_demand_data END"
    connection.execute(drop_table_query)

    create_table_query = f"CREATE TABLE historical_demand_data (Datetime DATETIME, Forecast_Pool_Price FLOAT, Actual_Posted_Pool_Price FLOAT, Forecast_AIL FLOAT, Actual_AIL FLOAT, Forecast_AIL_and_Actual_AIL_Difference FLOAT)"
    connection.execute(create_table_query)

def get_web_page_content(start_timestamp, end_timestamp):
    params = {
        "beginDate": start_timestamp.strftime('%m%d%Y'),
        "endDate": end_timestamp.strftime('%m%d%Y'),
        "contentType": "html"
    }

    try:
        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            return response.text
        else:
            print(f'Failed to retrieve data from the website. Status Code: {response.status_code}')
            
            return None
    except requests.RequestException as e:
        print(f'An error occurred during the request: {str(e)}')
        
        return None
    except Exception as e:
        print(f'An error occurred while fetching web page content: {str(e)}')
        
        return None

def parse_web_page_content(content):
    if content is not None:
        soup = BeautifulSoup(content, 'html.parser')
        table = soup.find('table', attrs={'border': '1', 'align': 'CENTER'})
        
        return table
    else:
        return None

def extract_data_from_table(table):
    if table is not None:
        data = []

        for row in table.find_all('tr'):
            columns = row.find_all('td')
            row_data = []

            for column in columns:
                field = column.text.strip()

                if '/' in field:
                    field = datetime.strptime(field.replace(' 24', ' 00'), '%m/%d/%Y %H')
                else:
                    try:
                        field = float(field.replace(',', ''))
                    except ValueError:
                        field = 0

                row_data.append(field)
            data.append(row_data)

        return data
    else:
        return None

def scrape_data_to_list(start_timestamp, end_timestamp):
    content = get_web_page_content(start_timestamp, end_timestamp)
    table = parse_web_page_content(content)
    data = extract_data_from_table(table)
    
    return data

def insert_data_to_azure_sql(connection, data, table_name):
    if data is not None:
        for row_data in data[1:]:
            sql = f"INSERT INTO {table_name} VALUES (?, ?, ?, ?, ?, ?)"
            connection.execute(sql, row_data)

        print(f'Data has been saved to {table_name} in Azure SQL Database.')

def fetch_data_from_azure_sql(connection, start_timestamp, end_timestamp, table_name):
    query = f"SELECT * FROM {table_name} WHERE Datetime >= ? AND Datetime <= ?"
    data = pd.read_sql_query(query, connection.conn, params=[start_timestamp, end_timestamp])

    return data

def preprocess_data(data, weather_data):
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    data = data.merge(weather_data, on='Datetime')
    data['Datetime'] = data['Datetime'].astype(int) // 10**9
    non_numeric_columns = data.select_dtypes(exclude=['number'])
    data = data.drop(non_numeric_columns, axis=1)

    return data

def train_and_predict_model(data):
    X = data.drop(['Actual_Posted_Pool_Price'], axis=1)
    y = data['Actual_Posted_Pool_Price']

    X_train = X.iloc[:-23]
    y_train = y.iloc[:-23]
    X_val = X.iloc[-23:]
    y_val = y.iloc[-23:]

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    mae = mean_absolute_error(y_val, y_pred)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    print(f'Predicted Demand for the Next 24 Hours: {y_pred}')
    print(f'Mean Absolute Error: {mae}')
    print(f'Root Mean Squared Error: {rmse}')


# Main Execution Flow
start_timestamp = datetime.now() - relativedelta(years=1)
end_timestamp = start_timestamp + timedelta(days=2)

connection = DatabaseConnection()
connection.connect()

create_azure_sql_table(connection)

scraped_data = scrape_data_to_list(start_timestamp, end_timestamp)
if scraped_data:
    insert_data_to_azure_sql(connection, scraped_data, 'historical_demand_data')

demand_data = fetch_data_from_azure_sql(connection, start_timestamp, end_timestamp, 'historical_demand_data')
weather_data = fetch_data_from_azure_sql(connection, start_timestamp, end_timestamp, 'calgary_weather_data')

preprocessed_data = preprocess_data(demand_data, weather_data)
train_and_predict_model(preprocessed_data)

# Disconnect from the database when done
connection.disconnect()





